# Tutorial 1.6: Framework Integrations

## Working with Multiple GenAI Frameworks

Welcome to framework integrations! MLflow supports 30+ GenAI frameworks, making it the most flexible platform for LLM development. This notebook shows you how to work with the most popular frameworks.

### What You'll Learn
- Overview of MLflow's framework integrations
- Working with OpenAI (direct API)
- Working with LangChain (chains and agents)
- Working with LlamaIndex (document indexing and RAG)
- Comparing frameworks and choosing the right one
- Best practices for each framework

### Prerequisites
- Completed previous notebooks (1.1-1.5)
- OpenAI API key or Databricks AI Gateway configured

### Estimated Time: 15-20 minutes

---
## Step 1: Framework Overview

### MLflow's 40+ Integrations

MLflow provides automatic tracing for:

**LLM Providers:**
- OpenAI, Anthropic, Cohere, Azure OpenAI
- AWS Bedrock, Google Vertex AI
- Ollama, vLLM, Together AI

**Frameworks:**
- LangChain, LlamaIndex, Haystack
- DSPy, AutoGen, CrewAI
- Guardrails AI, Phoenix

### Comparison Matrix

| Feature | OpenAI | LangChain | LlamaIndex |
|---------|--------|-----------|------------|
| **Complexity** | Low | Medium | Medium |
| **Learning Curve** | Easy | Medium | Medium |
| **Use Case** | Direct calls | Workflows | Doc Q&A |
| **Tracing** | ✅ Auto | ✅ Auto | ✅ Auto |
| **Agents** | Manual | ✅ Built-in | ✅ Built-in |
| **RAG** | Manual | ✅ Built-in | ✅ Built-in |
| **Customization** | Full | High | High |
| **Performance** | Fast | Medium | Medium |

### When to Use Each

**OpenAI Direct API:**
- Simple Q&A applications
- Maximum control over prompts
- Lowest latency
- Custom implementations

**LangChain:**
- Complex multi-step workflows
- Agent applications with tools
- Need for abstractions
- Rapid prototyping

**LlamaIndex:**
- Document-heavy applications
- Advanced indexing strategies
- Multiple data sources
- Knowledge management

---
## Step 2: Environment Setup

In [ ]:
# Install additional frameworks
!uv add langchain langchain-openai

print("✅ Frameworks installed")

In [ ]:
import mlflow
from dotenv import load_dotenv
from utils.clnt_utils import (
    is_databricks_client, 
    is_databricks_ai_gateway_client,
    get_databricks_ai_gateway_client,
    get_openai_client,
    get_ai_gateway_model_names
)

# Load environment
load_dotenv()

# Configure MLflow
mlflow.set_tracking_uri("http://localhost:5000")

# Determine provider and initialize client
use_databricks_provider = is_databricks_client()
use_databricks_ai_gateway = is_databricks_ai_gateway_client()

if use_databricks_ai_gateway:
    client = get_databricks_ai_gateway_client()
    model_name = get_ai_gateway_model_names()[0]
    provider_name = "Databricks AI Gateway"
else:
    client = get_openai_client()
    model_name = "gpt-5-mini"
    provider_name = "OpenAI"

print(f"✅ Environment configured: using {provider_name} client")
print(f"   MLflow version: {mlflow.__version__}")
print(f"   Tracking URI: {mlflow.get_tracking_uri()}")
print(f"   Model name: {model_name}")

# Enable OpenAI autologging
mlflow.openai.autolog()

mlflow.set_experiment("09-framework-integrations")

print("✅ OpenAI autologging: ENABLED")

---
## Step 3: OpenAI Framework (Already Covered)

We've been using OpenAI throughout this tutorial series.

### Quick Review

### OpenAI Strengths

- ✅ **Simplicity**: Direct API calls
- ✅ **Performance**: No abstraction overhead
- ✅ **Control**: Full control over prompts
- ✅ **Latest features**: Immediate access to new models
- ✅ **Documentation**: Extensive official docs

---
## Step 4: LangChain Framework

LangChain provides abstractions for building LLM applications.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Enable LangChain autologging
mlflow.langchain.autolog()

print("✅ LangChain autologging enabled")

In [ ]:
from utils.clnt_utils import (
    get_databricks_langchain_chat_client, 
    get_langchain_chat_openai_client,
    get_databricks_ai_gateway_langchain_client
)

# Simple LangChain chain
print("\n🔗 LangChain Example 1: Simple Chain\n")

# Create prompt template
prompt = ChatPromptTemplate.from_template(
    "You are a {role}. Answer: {question}"
)

# Create LLM based on provider
if use_databricks_ai_gateway:
    llm = get_databricks_ai_gateway_langchain_client(model_name, temperature=1.0)
elif use_databricks_provider:
    llm = get_databricks_langchain_chat_client(model_name, temperature=1.0)
else:
    llm = get_langchain_chat_openai_client(model_name, temperature=1.0)

# Create chain using LCEL (LangChain Expression Language)
chain = prompt | llm | StrOutputParser()

# Run chain (automatically traced!)
result = chain.invoke({
    "role": "MLflow expert",
    "question": "What makes LangChain different from using OpenAI directly?"
})

print(result)
print("\n✅ Chain execution fully traced!")
print("   - Prompt construction")
print("   - LLM call")
print("   - Output parsing")

### # More complex chain with multiple steps


In [ ]:
# More complex chain with multiple steps
print("\n🔗 LangChain Example 2: Multi-Step Chain\n")


# Step 1: Generate topic
topic_prompt = ChatPromptTemplate.from_template(
    "Generate a technical topic about {domain}"
)
topic_chain = topic_prompt | llm | StrOutputParser()

# Step 2: Create outline
outline_prompt = ChatPromptTemplate.from_template(
    "Create a 3-point outline for: {topic}"
)
outline_chain = outline_prompt | llm | StrOutputParser()

# Execute pipeline
topic = topic_chain.invoke({"domain": "MLOps"})
print(f"Topic: {topic}\n")

outline = outline_chain.invoke({"topic": topic})
print(f"Outline:\n{outline}")

print("\n✅ Multi-step chain traced!")
print("   Each chain creates separate spans")
print("   Full execution visible in MLflow UI")

### LangChain Strengths

- ✅ **Abstractions**: Reusable components
- ✅ **Chains**: Complex multi-step workflows
- ✅ **Agents**: Built-in agent patterns
- ✅ **Tools**: Easy tool integration
- ✅ **Community**: Large ecosystem

---
## Step 5: LlamaIndex Framework

LlamaIndex specializes in document indexing and retrieval-augmented generation (RAG).

In [ ]:
import os
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.llms.openai import OpenAI as LlamaIndexOpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Enable LlamaIndex autologging
mlflow.llama_index.autolog()

# Configure LlamaIndex Settings based on provider
# LlamaIndex defaults to OpenAI, so we need to explicitly configure it for Databricks AI Gateway
if use_databricks_ai_gateway:
    # Get Databricks AI Gateway credentials
    databricks_token = os.environ.get("DATABRICKS_TOKEN")
    ai_gateway_base_url = os.environ.get("AI_GATEWAY_BASE_URL")
    
    # Configure LLM for Databricks AI Gateway (OpenAI-compatible endpoint)
    Settings.llm = LlamaIndexOpenAI(
        model=os.environ.get("AI_GATEWAY_LLM_MODEL", "jsd-gpt-5-2"),
        api_key=databricks_token,
        api_base=ai_gateway_base_url
    )
    
    # Configure embedding model for Databricks AI Gateway
    # Use model_name (not model) to bypass OpenAIEmbeddingModelType enum validation
    Settings.embed_model = OpenAIEmbedding(
        model_name=os.environ.get("AI_GATEWAY_EMBED_MODEL", "jsd-text-embedding-3-small"),
        api_key=databricks_token,
        api_base=ai_gateway_base_url
    )
    print("✅ LlamaIndex configured for Databricks AI Gateway")
    print(f"   LLM: {Settings.llm.model}")
    print(f"   Embeddings: {Settings.embed_model.model_name}")
else:
    # Use default OpenAI settings (requires OPENAI_API_KEY)
    print("✅ LlamaIndex using default OpenAI configuration")

print("✅ LlamaIndex autologging enabled")

In [ ]:
# Create sample documents
print("\n📚 LlamaIndex Example: Document Q&A\n")

documents = [
    Document(text="MLflow is an open source platform for the complete ML lifecycle. It provides experiment tracking, model registry, and deployment capabilities."),
    Document(text="MLflow Tracing captures the complete execution of GenAI applications, including LLM calls, retrieval steps, and tool usage."),
    Document(text="MLflow integrates with 30+ frameworks including OpenAI, LangChain, LlamaIndex, and more."),
    Document(text="MLflow supports collaborative development with experiment sharing, prompt management, and model versioning."),
]

# Create index (automatically traced)
# Note: This uses the LLM and embedding model configured in Settings above
index = VectorStoreIndex.from_documents(documents)

# Create query engine
query_engine = index.as_query_engine()

# Query the index (automatically traced)
response = query_engine.query("What tracing capabilities does MLflow have?")

print(f"Query: What tracing capabilities does MLflow have?")
print(f"\nAnswer: {response}")

print("\n✅ LlamaIndex execution fully traced!")
print("   - Document indexing")
print("   - Query embedding")
print("   - Retrieval")
print("   - Response synthesis")

---
## Step 6: Best Practices

### OpenAI Best Practices

```python
# DO:
✅ Use structured prompts
✅ Implement retry logic
✅ Handle rate limits
✅ Stream responses for UX
✅ Cache results when possible

# DON'T:
❌ Hardcode prompts
❌ Ignore error responses
❌ Skip cost tracking
❌ Use synchronous calls in production
```

### LangChain Best Practices

```python
# DO:
✅ Use LCEL for chains
✅ Leverage built-in components
✅ Test chains independently
✅ Use async for better performance
✅ Enable debug mode during development

# DON'T:
❌ Over-abstract simple use cases
❌ Ignore performance overhead
❌ Skip error handling in chains
❌ Use deprecated components
```

### LlamaIndex Best Practices

```python
# DO:
✅ Choose appropriate index type for your use case
✅ Chunk documents thoughtfully
✅ Use metadata for filtering
✅ Cache embeddings when possible
✅ Monitor retrieval quality

# DON'T:
❌ Index without preprocessing
❌ Use default settings blindly
❌ Ignore index refresh strategies
❌ Skip evaluation of retrievals
```

---
## Summary

In this notebook, you learned:

1. ✅ Overview of MLflow's 30+ framework integrations
2. ✅ Working with OpenAI (direct API)
3. ✅ Working with LangChain (chains and workflows)
4. ✅ Working with LlamaIndex (document indexing and RAG)
5. ✅ Best practices for each framework

### Key Takeaways

- **All frameworks** are automatically traced by MLflow
- **Choose based on use case**, not hype
- **OpenAI** for simplicity and performance
- **LangChain** for complex workflows and agents
- **LlamaIndex** for document-heavy applications
- **Mix frameworks** when it makes sense

### Framework Comparison Summary

| Aspect | OpenAI | LangChain | LlamaIndex |
|--------|--------|-----------|------------|
| **Best For** | Simple apps | Workflows | Doc Q&A |
| **Performance** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ |
| **Ease of Use** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ |
| **Flexibility** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ |
| **RAG Support** | ⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ |
| **Agents** | ⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ |

### What's Next?

**📓 Notebook 1.7: Evaluating Agents**

Learn how to evaluate your GenAI applications:
- LLM-as-Judge evaluation patterns
- MLflow built-in scorers
- Custom scorers with @scorer decorator
- DeepEval integration

### Additional Resources

- [MLflow LangChain Integration](https://mlflow.org/docs/latest/llms/langchain/index.html)
- [MLflow LlamaIndex Integration](https://mlflow.org/docs/latest/llms/llama-index/index.html)
- [Framework Examples](https://github.com/mlflow/mlflow/tree/master/examples)

---
## Summary

In this notebook, you learned:

1. ✅ Overview of MLflow's integrations
2. ✅ Working with OpenAI (direct API)
3. ✅ Working with LangChain (chains and workflows)
4. ✅ Best practices for each framework

### Key Takeaways

- **All frameworks** are automatically traced by MLflow
- **Choose based on use case**, not hype
- **OpenAI** for simplicity and performance
- **LangChain** for complex workflows and agents
- **Mix frameworks** when it makes sense

### What's Next?

**📓 Notebook 1.7: Evaluating Agents**

Learn how to evaluate your GenAI applications:
- LLM-as-Judge evaluation patterns
- MLflow built-in scorers
- Custom scorers with @scorer decorator
- DeepEval integration

### Additional Resources

- [MLflow LangChain Integration](https://mlflow.org/docs/latest/llms/langchain/index.html)
- [Framework Examples](https://github.com/mlflow/mlflow/tree/master/examples)